In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.activations import relu, tanh
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.metrics import binary_accuracy
import tensorflow_datasets as tfds
from tensorflow_addons.layers import InstanceNormalization

import numpy as np
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')
print("Tensorflow", tf.__version__)
from packaging.version import parse as parse_version
# assert parse_version(tf.__version__) < parse_version("2.4.0"), \
#     f"Please install TensorFlow version 2.3.1 or older. Your current version is {tf.__version__}."

ImportError: cannot import name '_initialize_variables' from 'keras.src.backend' (c:\Users\Mahmoud\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\__init__.py)

# Try to get GPU power

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

## Input Pipeline
The input pipeline is adapted from https://www.tensorflow.org/tutorials/generative/cyclegan

In [ ]:
image_shape = (256, 256, 3)
IMG_HEIGHT = image_shape[0]
IMG_WIDTH = image_shape[1]

## preprocessing for data

In [ ]:
BATCH_SIZE = 1
BUFFER_SIZE = 1000
AUTOTUNE = tf.data.experimental.AUTOTUNE

dataset, ds_info = tfds.load('cycle_gan/horse2zebra', with_info=True,  as_supervised=True)
train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

def random_crop(image):
    cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image

# normalizing the images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_image_train(image, label):
    image = random_jitter(image)
    image = normalize(image)
    return image

def preprocess_image_test(image, label):
    image = normalize(image)
    return image

train_horses = train_horses.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).repeat()

train_zebras = train_zebras.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).repeat()

test_horses = test_horses.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(4).repeat()

test_zebras = test_zebras.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(4).repeat()

train_dataset = tf.data.Dataset.zip((train_horses, train_zebras))
test_dataset = tf.data.Dataset.zip((test_horses, test_zebras))

In [ ]:
class CycleGAN():
    #Initializes the class with an __init__ method which takes input_shape as a parameter.
    def __init__(self, input_shape):
        self.input_shape = input_shape
        # Defines two input layers image_A and image_B with the given input_shape.
        image_A = layers.Input(shape=input_shape)
        image_B = layers.Input(shape=input_shape)

        # discriminator
        #Builds discriminator networks (discriminator_A and discriminator_B) using the build_discriminator method.
        self.discriminator_B = self.build_discriminator()
        self.discriminator_A = self.build_discriminator()
        self.discriminator_B.trainable = False
        self.discriminator_A.trainable = False
        self.optimizer_discriminator_B = Adam(2e-4, 0.5)
        self.optimizer_discriminator_A = Adam(2e-4, 0.5)
        
        self.generator_AB = self.build_generator()
        self.generator_BA = self.build_generator()
        
        # forward
        fake_B = self.generator_AB(image_A)
        discriminator_B_output = self.discriminator_B(fake_B)
        reconstructed_A = self.generator_BA(fake_B)
        
        # backward
        fake_A = self.generator_BA(image_B)
        discriminator_A_output = self.discriminator_A(fake_A)
        reconstructed_B = self.generator_AB(fake_A)
        
        # identity
        # Generates identity mappings for images A and B using respective generators.
        identity_A = self.generator_AB(image_A)
        identity_B = self.generator_BA(image_B)
        # Defines the CycleGAN model with inputs as image_A and image_B, and outputs as discriminator outputs, reconstructed images, and identity mappings.
        self.model = Model(inputs=[image_A, image_B],
                           outputs=[discriminator_B_output, discriminator_A_output,
                                    reconstructed_A, reconstructed_B,
                                    identity_A, identity_B
                                    ])
        
        # build generator pipeline with frozen discriminator
        self.LAMBDA = 10
        self.LAMBDA_ID = 2
        # Compiles the model with multiple losses for discriminators and generators, and sets the Adam optimizer with specified learning rate and beta_1 parameters.

        self.model.compile(loss = ['mse','mse', 'mae','mae', 'mae','mae'],
                           optimizer = Adam(2e-4, 0.5),
                           loss_weights=[1, 1, 
                                         self.LAMBDA, self.LAMBDA,
                                         self.LAMBDA_ID, self.LAMBDA_ID])
        self.discriminator_B.trainable = True
        self.discriminator_A.trainable = True
        self.mse = tf.keras.losses.MeanSquaredError()
        
        self.patch_size = self.discriminator_B.output.shape[1]
        
    def mse_loss(self, y_true, y_pred):
        
        loss = self.mse(y_true, y_pred)

        return loss
    
    def downsample(self, channels, kernels, strides=2, norm=True, activation=True, dropout=False):
        initializer = tf.random_normal_initializer(0., 0.02)
        block = tf.keras.Sequential()
        block.add(layers.Conv2D(channels, kernels, strides=strides, padding='same', 
                                use_bias=False, kernel_initializer=initializer))

        if norm:
            block.add(InstanceNormalization())              
        if activation:
            block.add(layers.LeakyReLU(0.2)) 
        if dropout:
            block.add(layers.Dropout(0.5))

        return block

    def upsample(self, channels, kernels, strides=1, norm=True, activation=True, dropout=False):
        initializer = tf.random_normal_initializer(0., 0.02)
        block = tf.keras.Sequential()
        block.add(layers.UpSampling2D((2,2)))
        block.add(layers.Conv2D(channels, kernels, strides=strides, padding='same', 
                                use_bias=False, kernel_initializer=initializer))

        if norm:
            block.add(InstanceNormalization())              
        if activation:
            block.add(layers.LeakyReLU(0.2)) 
        if dropout:
            block.add(layers.Dropout(0.5))

        return block

    def build_generator(self):

        DIM = 64

        input_image = layers.Input(shape=image_shape)
        down1 = self.downsample(DIM, 4, norm=False)(input_image) # 128, DIM
        down2 = self.downsample(2*DIM, 4)(down1) # 64, 2*DIM
        down3 = self.downsample(4*DIM, 4)(down2) # 32, 4*DIM
        down4 = self.downsample(4*DIM, 4)(down3) # 16, 4*DIM
        down5 = self.downsample(4*DIM, 4)(down4) # 8, 4*DIM
        down6 = self.downsample(4*DIM, 4)(down5) # 4, 4*DIM
        down7 = self.downsample(4*DIM, 4)(down6) # 2, 4*DIM


        up6 = self.upsample(4*DIM, 4, dropout=True)(down7) # 4,4*DIM
        concat6 = layers.Concatenate()([up6, down6])   

        up5 = self.upsample(4*DIM, 4, dropout=True)(concat6) 
        concat5 = layers.Concatenate()([up5, down5]) 

        up4 = self.upsample(4*DIM, 4, dropout=True)(concat5) 
        concat4 = layers.Concatenate()([up4, down4]) 

        up3 = self.upsample(4*DIM, 4)(concat4) 
        concat3 = layers.Concatenate()([up3, down3]) 

        up2 = self.upsample(2*DIM, 4)(concat3) 
        concat2 = layers.Concatenate()([up2, down2]) 

        up1 = self.upsample(DIM, 4)(concat2) 
        concat1 = layers.Concatenate()([up1, down1]) 

        output_image = tanh(self.upsample(3, 4, norm=False, activation=False)(concat1))

        return Model(input_image, output_image)         
    
    def build_discriminator(self):
        DIM = 64
        
        input_image = layers.Input(shape=image_shape)
        x = self.downsample(DIM, 4, norm=False)(input_image) # 128
        x = self.downsample(2*DIM, 4)(x) # 64
        x = self.downsample(4*DIM, 4)(x) # 32
        x = self.downsample(8*DIM, 4, strides=1)(x) # 29
        output = layers.Conv2D(1, 4)(x)

        return Model(input_image, output)     
    
    def train_discriminator(self, direction, 
                            real_images_A, 
                            real_images_B):
        if direction=='AB':
            generator = self.generator_AB
            discriminator = self.discriminator_B
            optimizer = self.optimizer_discriminator_B
        else:
            generator = self.generator_BA
            discriminator = self.discriminator_A
            optimizer = self.optimizer_discriminator_A
            
        real_labels = tf.ones((self.batch_size, self.patch_size, self.patch_size, 1))
        fake_labels = tf.zeros((self.batch_size, self.patch_size, self.patch_size, 1))
                  
        fake_images = generator.predict(real_images_A)
        
        with tf.GradientTape() as gradient_tape:
            
            # forward pass
            pred_fake = discriminator(fake_images)
            pred_real = discriminator(real_images_B)
            
            # calculate losses
            loss_fake = self.mse_loss(fake_labels, pred_fake)
            loss_real = self.mse_loss(real_labels, pred_real)           
            
            # total loss
            total_loss = 0.5*(loss_fake + loss_real)
            
            # apply gradients
            gradients = gradient_tape.gradient(total_loss, discriminator.trainable_variables)
            
            optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))

        return total_loss
    
    def train(self, data_generator, test_data_generator, batch_size, steps, interval=100):
        checkpoint_path = "./checkpoints/cyclegan_horse2zebra"
        ckpt = tf.train.Checkpoint(gen_ab = self.generator_AB,
                                    gen_ba = self.generator_BA)
        ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=40)
        real_labels = tf.ones((batch_size, self.patch_size, self.patch_size, 1))
        self.batch_size = batch_size
        for i in range(steps):
            real_images_A, real_images_B = next(data_generator)

            d_loss_AB = self.train_discriminator("AB", real_images_A, real_images_B)
            d_loss_BA = self.train_discriminator("BA", real_images_B, real_images_A)    
            # train generator
            combined_loss = self.model.train_on_batch([real_images_A, real_images_B], 
                                                      [real_labels, real_labels,
                                                       real_images_A, real_images_B,
                                                       real_images_A, real_images_B
                                                       ])
            
            
            if i%interval == 0:
                ckpt_save_path = ckpt_manager.save()
                print ('Saving checkpoint for step {} at {}'.format(i,
                                                                     ckpt_save_path))
                msg = "Step {}: d_loss_AB {:.4f} d_loss_AB {:.4f} g_loss_AB {:.4f} g_loss_BA {:.4f} reconstr_loss_AB {:.4f} reconstr_loss_BA {:.4f}"\
                .format(i, d_loss_AB, d_loss_BA, combined_loss[0], combined_loss[1], combined_loss[2], combined_loss[3])
                print(msg)
                val_images = next(test_data_generator)        
                val_images_AB = self.generator_AB.predict(val_images[0])
                val_images_BA = self.generator_BA.predict(val_images[1])
                self.plot_images(val_images, val_images_AB, val_images_BA)
            
    def plot_images(self, real_images, images_AB, images_BA):   
        grid_row = min(images_AB.shape[0], 4)
        grid_col = 4
        f, axarr = plt.subplots(grid_row, grid_col, figsize=(grid_col*6, grid_row*6))

        for row in range(grid_row):
            
            ax = axarr if grid_row==1 else axarr[row]

            ax[0].imshow((real_images[0][row]+1)/2)
            ax[0].axis('off')
            ax[1].imshow((images_BA[row]+1)/2)
            ax[1].axis('off') 
            ax[2].imshow((real_images[1][row]+1)/2)
            ax[2].axis('off') 
            ax[3].imshow((images_AB[row]+1)/2)
            ax[3].axis('off') 
        plt.show()
        
    def sample_images(self, number):
        z = tf.random.normal((number, self.z_dim))
        images = self.generator.predict(z)
        self.plot_images(images)
        return images
    
cyclegan = CycleGAN(image_shape)


In [ ]:
cyclegan.train(iter(train_dataset), iter(test_dataset), BATCH_SIZE, 20000, 1000)

In [ ]:
'''
ckpt_path = "./checkpoints/cyclegan_horse2zebra/ckpt-25"
ckpt = tf.train.Checkpoint(gen_ab = cyclegan.generator_AB,
                           gen_ba = cyclegan.generator_BA).restore(ckpt_path)
'''

In [ ]:
test_iterator = iter(train_dataset)

In [ ]:
for _ in range(10):
    val_images = next(test_iterator) 
    val_images_AB = cyclegan.generator_AB.predict(val_images[0])
    val_images_BA = cyclegan.generator_BA.predict(val_images[1])
    cyclegan.plot_images(val_images, val_images_AB, val_images_BA)
